In [44]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("Spark").getOrCreate()
df = spark.read.csv("data (1).csv", header=True, inferSchema=True)
df.show()

+--------+-------------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------------+--------------------+
|children|      days_employed|dob_years|          education|education_id|   family_status|family_status_id|gender|income_type|debt|      total_income|             purpose|
+--------+-------------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------------+--------------------+
|       1| -8437.673027760233|       42|             высшее|           0| женат / замужем|               0|     F|  сотрудник|   0| 253875.6394525987|       покупка жилья|
|       1| -4024.803753850451|       36|            среднее|           1| женат / замужем|               0|     F|  сотрудник|   0|112080.01410244203|приобретение авто...|
|       0| -5623.422610230956|       33|            Среднее|           1| женат / замужем|               0|     M|  сотрудник|   0|145885.95

Импортируем необходимые бибилотеки, а также читаем и выводим исходный дата фрейм

In [45]:
df1 = df.select([count(when(col(c).contains('None') | col(c).contains('NULL') | (col(c) == '' ) | col(c).isNull() | isnan(c), c )).alias(c) for c in df.columns])
df1.show()

+--------+-------------+---------+---------+------------+-------------+----------------+------+-----------+----+------------+-------+
|children|days_employed|dob_years|education|education_id|family_status|family_status_id|gender|income_type|debt|total_income|purpose|
+--------+-------------+---------+---------+------------+-------------+----------------+------+-----------+----+------------+-------+
|       0|         2174|        0|        0|           0|            0|               0|     0|          0|   0|        2174|      0|
+--------+-------------+---------+---------+------------+-------------+----------------+------+-----------+----+------------+-------+


Во всем файле найдены пустые значения только в двух столбцах, которые содержат информацию о количестве отработанных дней а также о ежемесячном доходе, из чего можно сделать вывод о том, что человек сейчас не трудоустроен, т.е. не может получать за это зарплату

In [46]:
df2 = df.select([(count(when(col(c).contains('None') | col(c).contains('NULL') | (col(c) == '' ) | col(c).isNull() | isnan(c), c ))*100/count(lit(1))).alias(c) for c in df.columns])
df2.show()

+--------+------------------+---------+---------+------------+-------------+----------------+------+-----------+----+------------------+-------+
|children|     days_employed|dob_years|education|education_id|family_status|family_status_id|gender|income_type|debt|      total_income|purpose|
+--------+------------------+---------+---------+------------+-------------+----------------+------+-----------+----+------------------+-------+
|     0.0|10.099883855981417|      0.0|      0.0|         0.0|          0.0|             0.0|   0.0|        0.0| 0.0|10.099883855981417|    0.0|
+--------+------------------+---------+---------+------------+-------------+----------------+------+-----------+----+------------------+-------+


рассчет доли количества пропусков от общего количества значений в столбце в процентах.

In [47]:
df = df.withColumn("days_employed", F.abs('days_employed'))
days_employed_median = df.agg(F.median('days_employed')).collect()[0][0]
df = df.fillna({'days_employed': days_employed_median})
df = df.withColumn('days_employed', df['days_employed'].cast(IntegerType()))
df.show()

+--------+-------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------------+--------------------+
|children|days_employed|dob_years|          education|education_id|   family_status|family_status_id|gender|income_type|debt|      total_income|             purpose|
+--------+-------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------------+--------------------+
|       1|         8437|       42|             высшее|           0| женат / замужем|               0|     F|  сотрудник|   0| 253875.6394525987|       покупка жилья|
|       1|         4024|       36|            среднее|           1| женат / замужем|               0|     F|  сотрудник|   0|112080.01410244203|приобретение авто...|
|       0|         5623|       33|            Среднее|           1| женат / замужем|               0|     M|  сотрудник|   0|145885.95229686378|       покупка жилья|
|   

Заполнение данных медианным способом является лучшим решением для количественных переменных, так как на медиану не влияет посторонний "шум". К тому же, количество наших данных велико, поэтому медиана будет равняться среднему значению данных.
данный блок кода проверяет каждое значение в столбце days_employed, и если оно отрицательно, то возвращеает это же значение по модулю, а также заполняет пропуски медианным способом Возможно, отрицательность значений вызвана программным сбоем при заполнении значений

In [48]:
total_income_median = df.agg(F.median('total_income')).collect()[0][0]
df = df.fillna({'total_income': total_income_median})
df = df.withColumn('total_income', df['total_income'].cast(IntegerType()))
df.show()

+--------+-------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------+--------------------+
|children|days_employed|dob_years|          education|education_id|   family_status|family_status_id|gender|income_type|debt|total_income|             purpose|
+--------+-------------+---------+-------------------+------------+----------------+----------------+------+-----------+----+------------+--------------------+
|       1|         8437|       42|             высшее|           0| женат / замужем|               0|     F|  сотрудник|   0|      253875|       покупка жилья|
|       1|         4024|       36|            среднее|           1| женат / замужем|               0|     F|  сотрудник|   0|      112080|приобретение авто...|
|       0|         5623|       33|            Среднее|           1| женат / замужем|               0|     M|  сотрудник|   0|      145885|       покупка жилья|
|       3|         4124|       32|      

данный блок кода изменяет вещественный тип значений в столбце total_income на целочисленный, используя метод astype

In [49]:
df = df.drop_duplicates()
df.show()

+--------+-------------+---------+-------------------+------------+--------------------+----------------+------+-----------+----+------------+--------------------+
|children|days_employed|dob_years|          education|education_id|       family_status|family_status_id|gender|income_type|debt|total_income|             purpose|
+--------+-------------+---------+-------------------+------------+--------------------+----------------+------+-----------+----+------------+--------------------+
|       0|          145|       32|             высшее|           0|     женат / замужем|               0|     M|  компаньон|   0|      188982|покупка жилой нед...|
|       0|         1031|       50|            среднее|           1|     женат / замужем|               0|     F|  сотрудник|   0|      125165|               жилье|
|       2|          406|       32|             высшее|           0|           в разводе|               3|     F|госслужащий|   0|      318047|покупка жилой нед...|
|       0|      

используя метод drop_duplicates избавляемся от строк-дубликатов. Вероятно, дубликаты появились при объединении нескольких фреймов данных.

In [50]:
df = df.withColumn('education', initcap(col('education')))
df.show()

+--------+-------------+---------+-------------------+------------+--------------------+----------------+------+-----------+----+------------+--------------------+
|children|days_employed|dob_years|          education|education_id|       family_status|family_status_id|gender|income_type|debt|total_income|             purpose|
+--------+-------------+---------+-------------------+------------+--------------------+----------------+------+-----------+----+------------+--------------------+
|       0|          145|       32|             Высшее|           0|     женат / замужем|               0|     M|  компаньон|   0|      188982|покупка жилой нед...|
|       0|         1031|       50|            Среднее|           1|     женат / замужем|               0|     F|  сотрудник|   0|      125165|               жилье|
|       2|          406|       32|             Высшее|           0|           в разводе|               3|     F|госслужащий|   0|      318047|покупка жилой нед...|
|       0|      

В данном блоке кода мы выбираем столбец education и используя метод str.capitalize приводим значения в нем к одному регистру, а именно - первая буква заглавная, остальные строчные. Также в этом же блоке

In [51]:
dfED = df.select('education', 'education_id')
dfED = dfED.drop_duplicates().sort(['education_id'])
dfED.show()

+-------------------+------------+
|          education|education_id|
+-------------------+------------+
|             Высшее|           0|
|            Среднее|           1|
|Неоконченное Высшее|           2|
|          Начальное|           3|
|     Ученая Степень|           4|
+-------------------+------------+


In [52]:
dfFAM_STAT = df.select('family_status', 'family_status_id')
dfFAM_STAT = dfFAM_STAT.drop_duplicates().sort(['family_status_id'])
dfFAM_STAT.show()

+--------------------+----------------+
|       family_status|family_status_id|
+--------------------+----------------+
|     женат / замужем|               0|
|    гражданский брак|               1|
|      вдовец / вдова|               2|
|           в разводе|               3|
|Не женат / не зам...|               4|
+--------------------+----------------+


В двух предыдущих блоках мы создали два новых датафрейма, в которых хранятся значения уровня образования и его id в первом и статус семьи и его id во втором соответственно

In [53]:
df = df.drop('education','family_status')
df.show()

+--------+-------------+---------+------------+----------------+------+-----------+----+------------+--------------------+
|children|days_employed|dob_years|education_id|family_status_id|gender|income_type|debt|total_income|             purpose|
+--------+-------------+---------+------------+----------------+------+-----------+----+------------+--------------------+
|       0|          145|       32|           0|               0|     M|  компаньон|   0|      188982|покупка жилой нед...|
|       0|         1031|       50|           1|               0|     F|  сотрудник|   0|      125165|               жилье|
|       2|          406|       32|           0|               3|     F|госслужащий|   0|      318047|покупка жилой нед...|
|       0|          342|       42|           0|               0|     F|  сотрудник|   0|      151655|   операции с жильем|
|       2|         2194|       35|           1|               0|     M|  сотрудник|   0|      145017|операции с недвиж...|
|       0|      

удаляем столбцы education и family_status

In [54]:
df = df.withColumn('total_income_category', when ((df.total_income <= 30000), lit('E'))
                   .when((df.total_income >= 30001) & (df.total_income <= 50000), lit('D'))
                   .when((df.total_income >= 50001) & (df.total_income <= 200000), lit('C'))
                   .when((df.total_income >= 200001) & (df.total_income <= 1000000), 'B')
                   .otherwise(lit('A')))
df.show()

+--------+-------------+---------+------------+----------------+------+-----------+----+------------+--------------------+---------------------+
|children|days_employed|dob_years|education_id|family_status_id|gender|income_type|debt|total_income|             purpose|total_income_category|
+--------+-------------+---------+------------+----------------+------+-----------+----+------------+--------------------+---------------------+
|       0|          145|       32|           0|               0|     M|  компаньон|   0|      188982|покупка жилой нед...|                    C|
|       0|         1031|       50|           1|               0|     F|  сотрудник|   0|      125165|               жилье|                    C|
|       2|          406|       32|           0|               3|     F|госслужащий|   0|      318047|покупка жилой нед...|                    B|
|       0|          342|       42|           0|               0|     F|  сотрудник|   0|      151655|   операции с жильем|        

Разделяем категории заработка

In [55]:
df = df.withColumn('purpose_category', when ((df.purpose.contains('авто')), lit('Операции с автомобилем'))
                   .when((df.purpose.contains('недвиж') | df.purpose.contains('жил')), lit('Операции с недвижимостью'))
                   .when((df.purpose.contains('свадьб')), lit('Проведение свадьбы'))
                   .otherwise(lit('Получение образования')))
df.show()

+--------+-------------+---------+------------+----------------+------+-----------+----+------------+--------------------+---------------------+--------------------+
|children|days_employed|dob_years|education_id|family_status_id|gender|income_type|debt|total_income|             purpose|total_income_category|    purpose_category|
+--------+-------------+---------+------------+----------------+------+-----------+----+------------+--------------------+---------------------+--------------------+
|       0|          145|       32|           0|               0|     M|  компаньон|   0|      188982|покупка жилой нед...|                    C|Операции с недвиж...|
|       0|         1031|       50|           1|               0|     F|  сотрудник|   0|      125165|               жилье|                    C|Операции с недвиж...|
|       2|          406|       32|           0|               3|     F|госслужащий|   0|      318047|покупка жилой нед...|                    B|Операции с недвиж...|
|   

Приводим цели получения кредита к общему виду